In [ ]:
# Import libraries
import pandas as pd

# Load cleaned Netflix dataset
netflix_df = pd.read_csv('../data/clean/cleaned_netflix_data_with_platform.csv')

# Load cleaned Amazon dataset
amazon_df = pd.read_csv('../data/clean/cleaned_prime_data.csv')

In [ ]:
# Check columns to ensure alignment
print("Netflix columns:", netflix_df.columns)
print("Amazon columns:", amazon_df.columns)

In [ ]:
# Combine datasets
combined_df = pd.concat([netflix_df, amazon_df], ignore_index=True)

In [ ]:
# Quick check
print(combined_df.shape)
print(combined_df['platform'].value_counts())

In [ ]:
# Export combined dataset
combined_df.to_csv('../data/clean/combined_cleaned_data.csv', index=False)
print("Combined!")


In [ ]:
combined_df = pd.read_csv('../data/clean/combined_cleaned_data.csv')
combined_df.info()
combined_df.head()
